# Numpy Chebyshev Class

## 1. A Summary of the Problem

The issue at hand is that to create the NestedBasisFunctionSet object given a value for exactness, 3 things must be created:
1. basis_functions = a list of ChebyshevFirstKind objects of order 0 to the max polynomial order for the given exactness
2. levels = a nested list of the smolyak indices where indices are separated by the level they were added on
3. points = a list of extrema where each extrema is associated with a smolyak index indicated by the extremum's position in the list. These extrema are ordered in increasing level, and extrema added on the same level are ordered least to greatest. It is noted that for the nested extrema of Chebyshev polynomials of the 1st kind, the number of ChebyshevFirstKind objects needed is the same as the number of points.

The following table gives a summary of the things that must be calculated:

| Exactness | Max Polynomial Order | # Points Added | Smolyak Indices | Extrema Added at each Level |
|:-:|:-:|:-:|:-:|:-:|
| 0 | 0 | 1 | 0 | 0 |
| 1 | 2 | 2 | 1, 2 | -1, 1 |
| 2 | 4 | 2 |3, 4 | -0.7071,  0.7071 |
| 3 | 8 | 4 |5, 6, 7, 8 | -0.9239, -0.3827,  0.3827,  0.9239 |
| 4 | 16 | 8 |9, 10, 11, 12, 13, 14, 15, 16 | -0.9808, -0.8315, -0.5556, -0.1951,  0.1951,  0.5556,  0.8315,  0.9808 |

## 2. Old Method

To get the extrema, one can simply calculate all the extrema of the last polynomial in basis_functions using this method:

In [96]:
import math

import numpy

numpy.set_printoptions(formatter={'float': '{: 0.4f}'.format})

def compute_points(n):
    if n == 0:
        return [0]
    else:
        i = numpy.arange(n+1)
        return -numpy.cos(numpy.pi*i/n)
    
for i in [0,2,4,8]:
    print('Order: ' + str(i) + ' Extrema: '+str(compute_points(i)))

Order: 0 Extrema: [0]
Order: 2 Extrema: [-1.0000 -0.0000  1.0000]
Order: 4 Extrema: [-1.0000 -0.7071 -0.0000  0.7071  1.0000]
Order: 8 Extrema: [-1.0000 -0.9239 -0.7071 -0.3827 -0.0000  0.3827  0.7071  0.9239  1.0000]


The maximum polynomial required for a given exactness will have extrema at all the required points. This is because the polynomials of order 0, 2, 4, 8 and so on are nested, so for example 8 has all the extrema order 4 has plus extra, and order 4 has all of order 2's extrema, etc. 

However, these extrema are calculated from least to greatest, which isn't the correct order. To get the right order, the extrema of the largest order polynomial for each level are calculated in ascending order, and any extremum calculated that isn't already on the list is added. This means you have to be careful to make sure not to add extrema already in the list.

## 3. Numpy's Chebyshev Class

Numpy has a function called chebpts1 that can calculate some of these extrema. The raw code this function uses is linked here: https://github.com/numpy/numpy/blob/v1.23.0/numpy/polynomial/chebyshev.py#L1920-L1954

To understand how my code ends up using this function, let's do some initial investigation to just see what this function prints out.

In [97]:
for i in range(1,9):
    print('answer for input ' + str(i) + ' is ', end='')
    print(numpy.polynomial.chebyshev.chebpts1(i))


answer for input 1 is [ 0.0000]
answer for input 2 is [-0.7071  0.7071]
answer for input 3 is [-0.8660  0.0000  0.8660]
answer for input 4 is [-0.9239 -0.3827  0.3827  0.9239]
answer for input 5 is [-0.9511 -0.5878  0.0000  0.5878  0.9511]
answer for input 6 is [-0.9659 -0.7071 -0.2588  0.2588  0.7071  0.9659]
answer for input 7 is [-0.9749 -0.7818 -0.4339  0.0000  0.4339  0.7818  0.9749]
answer for input 8 is [-0.9808 -0.8315 -0.5556 -0.1951  0.1951  0.5556  0.8315  0.9808]


The function has one input, which is not actually related to the order of the polynomials these extrema come from. Instead, the input specifics how many points the function should print. The function will give an error if the input is 0 or not a whole number. Here is a table showing the outputs of chebpts1 that are worth noting.

| Input | chebpts1 Output | Exactness Where These Points Are Added | Polynomial Order Extrema First Appear |
|:-:|:-:|:-:|:-:|
| **1** | **0** | **0** | **0** |
| **2** | **-0.7071,  0.7071** | **2** | **4** |
| 3 | -0.8660,  0.0000,  0.8660 | NA | 6 |
| **4** | **-0.9239, -0.3827,  0.3827,  0.9239** | **3** | **8** |
| 5 | -0.9511, -0.5878,  0.0000,  0.5878,  0.9511 | NA | 10 |
| 6 | -0.9659, -0.7071, -0.2588,  0.2588,  0.7071,  0.9659 | NA | 12 |
| 7 | -0.9749, -0.7818, -0.4339,  0.0000,  0.4339,  0.7818,  0.9749 | NA | 14 |
| **8** | **-0.9808, -0.8315, -0.5556, -0.1951,  0.1951,  0.5556,  0.8315,  0.9808** | **4** | **16** |

As input increases onward to 16, 32, 64, etc. the pattern seems to be that it gives the extrema added by the polynomial of order n/2 where n is the input to chebpts1. So, to get the points added to the list by the polynomials, simply take the order of the polynomial, divide it by 2, and then plug that into chebpts1 to get the new extrema, no double checking for duplicates required. When looking at the number of points added each level, it is usually half the max polynomial order, so this reasoning makes sense. The expections are when the exactness is 0 or 1.

One major problem with using chebpts1 in the code, which is that the extrema -1 and 1 are nowhere to be found among chebpts1's outputs. This probably has to do with the fact that the way chebpts1 is set up means it can't give outputs of the same length for two different inputs. Looking at the table, exactness = 1 and exactness = 2 add the same number of points to the total, so it isn't capable of outputting [-1, 1] and [-0.7071, 0.7071]. To work around this, an exception needs to be made for ChebyshevFirstKind objects when n = 2. Then, n = 4 can return [-0.7071, 0.7071].

Another exception must be made when n = 0. The function chebpts1 can't accept 0 as an input since that would mean returning no points.

In [98]:
def points_with_numpy(n):
    """Compute extrema of Chebyshev polynomial of the first kind"""
    if n == 2:
        return [-1,1]
    elif n <= 0:
        return [0]
    else:
        num_points = math.ceil(n/2)
        return numpy.polynomial.chebyshev.chebpts1(num_points)

Here the exception is made for n = 2 and n = 0 such that even if n is less than zero there won't be an error. To get the right input for chebpts1, n is divided by 2. The ceil function is applied to prevent errors if n is odd. Odd values of n won't give their own extrema, but that doesn't matter since those values will never be used. Originally, it was intended that n = 1 would return 0 so that points could be taken from polynomials of order 2^i where i is bounded between 0 and the exactness inclusive. However, this ended up not coming to fruition in the function make_nested_set.

Speaking of make_nested_set...

In [99]:
def make_nested_set(exactness):
    """make_nested_set modified for demostration purposes"""
    
    # initialize variables
    levels = []
    points = []
    
    # Create bounds for the smolyak indices on each level.
    # Conveniently, the values for end_level_index are the same as the max polynomial degree for each level.
    end_level_index = [2**n for n in range(exactness+1)]
    end_level_index[0] = 0 # end_level_index values follow a 2^x scheme except that it is 0 when x is 0
    start_level_index = [0]
    start_level_index.extend([n+1 for n in end_level_index])
    
    # Create the list of basis functions.
    # Since the equation 2**exactness cannot be relied on to give the right value for max_degree when exactness = 0,
    # max_degree is set to the last value in end_level_index to avoid this problem
    max_degree = end_level_index[-1]
    basis_functions = [n for n in range(max_degree+1)]
    
    # add points and smolyak indices
    for i in range(0,exactness+1):
        levels.append(list(range(start_level_index[i],end_level_index[i]+1)))
        points.extend(points_with_numpy(basis_functions[end_level_index[i]]))
        
    print("For exactness " + str(exactness)+":")
    print("Orders of polynomials are from " + str(basis_functions[0]) + " to " + str(basis_functions[-1]))
    print("Levels are " + str(levels))
    print("Points are " + str(numpy.array(points)))

for i in range(5):
    make_nested_set(i)

For exactness 0:
Orders of polynomials are from 0 to 0
Levels are [[0]]
Points are [0]
For exactness 1:
Orders of polynomials are from 0 to 2
Levels are [[0], [1, 2]]
Points are [ 0 -1  1]
For exactness 2:
Orders of polynomials are from 0 to 4
Levels are [[0], [1, 2], [3, 4]]
Points are [ 0.0000 -1.0000  1.0000 -0.7071  0.7071]
For exactness 3:
Orders of polynomials are from 0 to 8
Levels are [[0], [1, 2], [3, 4], [5, 6, 7, 8]]
Points are [ 0.0000 -1.0000  1.0000 -0.7071  0.7071 -0.9239 -0.3827  0.3827  0.9239]
For exactness 4:
Orders of polynomials are from 0 to 16
Levels are [[0], [1, 2], [3, 4], [5, 6, 7, 8], [9, 10, 11, 12, 13, 14, 15, 16]]
Points are [ 0.0000 -1.0000  1.0000 -0.7071  0.7071 -0.9239 -0.3827  0.3827  0.9239
 -0.9808 -0.8315 -0.5556 -0.1951  0.1951  0.5556  0.8315  0.9808]
